# Alpaca-py options trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/options-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with options trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for options as an example.

In [ ]:
! python3 -m install alpaca-py
! python3 -m pip install alpaca-py

In [1]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/

from dotenv import load_dotenv
import os

# keys is stored in config.env file
# - Specify the path to the config.env file
dotenv_path = os.path.join(os.getcwd(), "config.env")
# Load environment variables from the specified .env file
load_dotenv(dotenv_path)

api_key = os.getenv("ALPACA_API_KEY_PAPER")
secret_key = os.getenv('ALPACA_API_SECRET_KEY_PAPER')
base_url = os.getenv('ALPACA_BASE_URL')

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

In [2]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.option import OptionDataStream

from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest    
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    GetAssetsRequest,
    MarketOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus 
)
from alpaca.common.exceptions import APIError

In [3]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [4]:
# check version of alpaca-py
alpaca.__version__

'0.30.1'

# Trading Client

In [5]:
# setup clients
# - TradingClient is a class. You can instantiate it given parameters such as API key and secret key.
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [6]:
# check trading account
# There are trhee new columns in the account object:
# - options_buying_power
# - options_approved_level
# - options_trading_level
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3P9DF5CZ5G',
    'accrued_fees': '0',
    'buying_power': '131923.96',
    'cash': '18061.62',
    'created_at': datetime.datetime(2024, 5, 15, 18, 59, 39, 40242, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '113862.34',
    'id': UUID('8124b3d1-e06e-4212-8930-0f9b5f69c050'),
    'initial_margin': '47900.36',
    'last_equity': '110604.17',
    'last_maintenance_margin': '30974.97',
    'long_market_value': '95800.72',
    'maintenance_margin': '32277.02',
    'multiplier': '2',
    'non_marginable_buying_power': '63961.98',
    'options_approved_level': 2,
    'options_buying_power': '81585.32',
    'options_trading_level': 2,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '113862.34',
    'regt_buying_power': '131923.96',
    's

In [7]:
# check account configuration
# - we have new field `max_options_trading_level`
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [8]:
# get list of assets which are options enabled
# - we can filter assets by `options_enabled` attribute
# - asset object has `options_enabled` attribute if it is options enabled
req = GetAssetsRequest(
  attributes = "options_enabled"  
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('b64671e3-d25c-47a9-bc11-60906bddf711'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'ELYS BMG GROUP, INC. Common Stock',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'ELYS',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('6358e6de-566c-4978-87ac-7aeed11e0945'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'Remark Hol

In [9]:
# get list of options contracts for the given underlying symbol (e.g. SPY,AAPL)
# - get_option_contracts() is a new method to get list of options contracts
# - in this example, we get 2 options contracts for SPY,AAPL
# - you can continue to fetch options contracts by specifying page_token from next_page_token of response
underlying_symbols = ["SPY", "AAPL"]
# - GetOptionContractsRequest is a request object (request model) to get list of options contracts
req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,               # specify underlying symbols
    status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
    expiration_date = None,                                # specify expiration date (specified date + 1 day range)
    expiration_date_gte = None,                            # we can pass date object
    expiration_date_lte = None,                            # or string (YYYY-MM-DD)
    root_symbol = None,                                    # specify root symbol
    type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
    style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
    strike_price_gte = None,                               # specify strike price range
    strike_price_lte = None,                               # specify strike price range
    limit = 2,                                             # specify limit
    page_token = None,                                     # specify page token
)
res = trade_client.get_option_contracts(req)
res

{   'next_page_token': 'Mg==',
    'option_contracts': [   {   'close_price': '225.72',
                                'close_price_date': datetime.date(2024, 7, 11),
                                'expiration_date': datetime.date(2024, 11, 15),
                                'id': 'fc7aa684-bdc4-405b-93bb-b095c58c05a3',
                                'name': 'AAPL Nov 15 2024 5 Call',
                                'open_interest': None,
                                'open_interest_date': None,
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 5.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL241115C00005000',
                                'tradable': True,
                                'type': <ContractType.CALL: 'call'>,


In [10]:
# continue to fetch option contracts if there is next_page_token in response
if res.next_page_token is not None:
    req = GetOptionContractsRequest(
        underlying_symbols = underlying_symbols,               # specify underlying symbols
        status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
        expiration_date = None,                                # specify expiration date (specified date + 1 day range)
        expiration_date_gte = None,                            # we can pass date object
        expiration_date_lte = None,                            # or string (YYYY-MM-DD)
        root_symbol = None,                                    # specify root symbol
        type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT) Default is "call"
        style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
        strike_price_gte = None,                               # specify strike price range
        strike_price_lte = None,                               # specify strike price range
        limit = 2,                                             # specify limit
        page_token = res.next_page_token,                      # specify page token
    )
    res = trade_client.get_option_contracts(req)
    display(res)

{   'next_page_token': 'NA==',
    'option_contracts': [   {   'close_price': '212.95',
                                'close_price_date': datetime.date(2024, 10, 10),
                                'expiration_date': datetime.date(2024, 11, 15),
                                'id': 'a3cbcd4c-aeb6-40e1-b871-3cbf18da88cf',
                                'name': 'AAPL Nov 15 2024 15 Call',
                                'open_interest': None,
                                'open_interest_date': None,
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 15.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL241115C00015000',
                                'tradable': True,
                                'type': <ContractType.CALL: 'call'

In [11]:
# get options contract by symbol
# - get_option_contract() is a new method to get options contract by symbol or id
# - in this example, we get the first options contract from the previous response
symbol = res.option_contracts[0].symbol
contract = trade_client.get_option_contract(symbol)
contract

{   'close_price': '212.95',
    'close_price_date': datetime.date(2024, 10, 10),
    'expiration_date': datetime.date(2024, 11, 15),
    'id': 'a3cbcd4c-aeb6-40e1-b871-3cbf18da88cf',
    'name': 'AAPL Nov 15 2024 15 Call',
    'open_interest': None,
    'open_interest_date': None,
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 15.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL241115C00015000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [12]:
# get options contract by id
# - in this example, we get the first options contract from the previous response
id = res.option_contracts[0].id
contract = trade_client.get_option_contract(symbol_or_id=id)
contract

{   'close_price': '212.95',
    'close_price_date': datetime.date(2024, 10, 10),
    'expiration_date': datetime.date(2024, 11, 15),
    'id': 'a3cbcd4c-aeb6-40e1-b871-3cbf18da88cf',
    'name': 'AAPL Nov 15 2024 15 Call',
    'open_interest': None,
    'open_interest_date': None,
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 15.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL241115C00015000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [27]:
# get put options contracts
# - in this example, we get put options contracts for SPY = SPDR S&P 500 ETF
underlying_symbols = ["SPY"]

# specify expiration date range
now = datetime.now(tz = ZoneInfo("America/New_York"))
day1 = now + timedelta(days = 1)
day60 = now + timedelta(days = 60)

req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,                     # specify underlying symbols
    status = AssetStatus.ACTIVE,                                 # specify asset status: active (default)
    expiration_date = None,                                      # specify expiration date (specified date + 1 day range)
    expiration_date_gte = day1.date(),                           # we can pass date object
    expiration_date_lte = day60.strftime(format = "%Y-%m-%d"),   # or string
    root_symbol = None,                                          # specify root symbol
    type = "put",                                                # specify option type: put
    style = ExerciseStyle.AMERICAN,                              # specify option style: american
    strike_price_gte = None,                                     # specify strike price range
    strike_price_lte = None,                                     # specify strike price range
    limit = 100,                                                 # specify limit
    page_token = None,                                           # specify page
)
res = trade_client.get_option_contracts(req)
res.option_contracts[:5]

[{   'close_price': None,
     'close_price_date': None,
     'expiration_date': datetime.date(2024, 11, 12),
     'id': '9143297e-3ea8-4e58-b629-bbda868e8680',
     'name': 'SPY Nov 12 2024 300 Put',
     'open_interest': None,
     'open_interest_date': None,
     'root_symbol': 'SPY',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,
     'strike_price': 300.0,
     'style': <ExerciseStyle.AMERICAN: 'american'>,
     'symbol': 'SPY241112P00300000',
     'tradable': True,
     'type': <ContractType.PUT: 'put'>,
     'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'underlying_symbol': 'SPY'},
 {   'close_price': None,
     'close_price_date': None,
     'expiration_date': datetime.date(2024, 11, 12),
     'id': '8ccda5f2-4aa5-4dd0-897e-1fef5b13813d',
     'name': 'SPY Nov 12 2024 305 Put',
     'open_interest': None,
     'open_interest_date': None,
     'root_symbol': 'SPY',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,


In [14]:
# get high open_interest contract
open_interest = 0
high_open_interest_contract = None
for contract in res.option_contracts:
    if (contract.open_interest is not None) and (int(contract.open_interest) > open_interest):
        open_interest = int(contract.open_interest)
        high_open_interest_contract = contract
high_open_interest_contract

{   'close_price': '0.03',
    'close_price_date': datetime.date(2024, 11, 8),
    'expiration_date': datetime.date(2024, 11, 12),
    'id': '537c92be-2e32-49c3-a5bd-fea5495a108b',
    'name': 'SPY Nov 12 2024 540 Put',
    'open_interest': '12749',
    'open_interest_date': datetime.date(2024, 11, 7),
    'root_symbol': 'SPY',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 540.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'SPY241112P00540000',
    'tradable': True,
    'type': <ContractType.PUT: 'put'>,
    'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'underlying_symbol': 'SPY'}

In [19]:
# place buy put option order
# - we can place buy put option order same as buy stock/crypto order
req = MarketOrderRequest(
    symbol = high_open_interest_contract.symbol,
    qty = 10,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('50d52dc3-87ac-475b-87b0-1684827f8a1e'),
    'canceled_at': None,
    'client_order_id': '2baf0122-426e-4ec7-8189-6d9c54ed401b',
    'created_at': datetime.datetime(2024, 9, 26, 15, 38, 9, 802510, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('9e20b47d-1e56-4d99-92ca-2750e07d4b4e'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': '10',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 9, 26, 15, 38, 9, 800254, tzinfo=TzInfo(UTC)),
    'symbol': 'SPY240

In [15]:
# get list of orders by specifying option contract symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [high_open_interest_contract.symbol],
    limit = 2,
)
orders = trade_client.get_orders(req)
orders

[]

In [16]:
# below cells should be done after market open otherwise there is no position for the option contract

# get positions filtered by option contract symbol
# if you do this example outside of market hours, you will see empty list
# because we have no position in this option contract
# please wait market open and run this example again
positions = trade_client.get_all_positions()
[pos for pos in positions if pos.symbol == high_open_interest_contract.symbol]

[]

In [20]:
# get positions by symbol
trade_client.get_open_position(symbol_or_asset_id=high_open_interest_contract.symbol)


{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('50d52dc3-87ac-475b-87b0-1684827f8a1e'),
    'asset_marginable': True,
    'avg_entry_price': '0.01',
    'avg_entry_swap_rate': None,
    'change_today': '0',
    'cost_basis': '10',
    'current_price': '0.01',
    'exchange': <AssetExchange.EMPTY: ''>,
    'lastday_price': '0.01',
    'market_value': '10',
    'qty': '10',
    'qty_available': '10',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY240927P00420000',
    'unrealized_intraday_pl': '0',
    'unrealized_intraday_plpc': '0',
    'unrealized_pl': '0',
    'unrealized_plpc': '0',
    'usd': None}

In [21]:
# get positions by contract id
trade_client.get_open_position(symbol_or_asset_id = high_open_interest_contract.id)

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('50d52dc3-87ac-475b-87b0-1684827f8a1e'),
    'asset_marginable': True,
    'avg_entry_price': '0.01',
    'avg_entry_swap_rate': None,
    'change_today': '0',
    'cost_basis': '10',
    'current_price': '0.01',
    'exchange': <AssetExchange.EMPTY: ''>,
    'lastday_price': '0.01',
    'market_value': '10',
    'qty': '10',
    'qty_available': '10',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY240927P00420000',
    'unrealized_intraday_pl': '0',
    'unrealized_intraday_plpc': '0',
    'unrealized_pl': '0',
    'unrealized_plpc': '0',
    'usd': None}

In [22]:
# close the option position (sell the option contract)
trade_client.close_position(
    symbol_or_asset_id = high_open_interest_contract.symbol,
    close_options = ClosePositionRequest(qty = "20")
)

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('50d52dc3-87ac-475b-87b0-1684827f8a1e'),
    'canceled_at': None,
    'client_order_id': '80cd04ed-b3df-4d98-9ae5-72610096ca9b',
    'created_at': datetime.datetime(2024, 9, 26, 15, 38, 16, 269122, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('796b7535-4b43-4848-9052-a6578060ef44'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': '10',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 9, 26, 15, 38, 16, 267234, tzinfo=TzInfo(UTC)),
    'symbol': 'SP

In [ ]:
# exercise the options position
# - this method does not return anything
trade_client.exercise_options_position(
    symbol_or_contract_id = high_open_interest_contract.symbol
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

fyi. you can open this notebook in another window and run below cell to check trade updates.

In [23]:
# ensure the event loop can run in a Jupyter notebook
nest_asyncio.apply()

# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

keyboard interrupt, bye


data transfer failed
Traceback (most recent call last):
  File "/Users/satoshiido/miniforge3/envs/python310/lib/python3.10/site-packages/websockets/legacy/protocol.py", line 968, in transfer_data
    message = await self.read_message()
  File "/Users/satoshiido/miniforge3/envs/python310/lib/python3.10/site-packages/websockets/legacy/protocol.py", line 1038, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
  File "/Users/satoshiido/miniforge3/envs/python310/lib/python3.10/site-packages/websockets/legacy/protocol.py", line 1113, in read_data_frame
    frame = await self.read_frame(max_size)
  File "/Users/satoshiido/miniforge3/envs/python310/lib/python3.10/site-packages/websockets/legacy/protocol.py", line 1170, in read_frame
    frame = await Frame.read(
  File "/Users/satoshiido/miniforge3/envs/python310/lib/python3.10/site-packages/websockets/legacy/framing.py", line 69, in read
    data = await reader(2)
  File "/Users/satoshiido/miniforge3/envs/python31

# Market Data (Historical)

In [15]:
# setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [35]:
# get options historical bars by symbol
req = OptionBarsRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    timeframe = TimeFrame(amount = 1, unit = TimeFrameUnit.Hour),   # specify timeframe
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                                # specify end datetime, default=now
    limit = 10,                                                      # specify limit
)
option_historical_data_client.get_option_bars(req).df

open  high   low  close  \
symbol             timestamp                                            
SPY241112P00540000 2024-11-07 16:00:00+00:00  0.07  0.07  0.07   0.07   
                   2024-11-07 19:00:00+00:00  0.06  0.06  0.06   0.06   
                   2024-11-07 20:00:00+00:00  0.06  0.06  0.06   0.06   
                   2024-11-08 14:00:00+00:00  0.04  0.04  0.04   0.04   
                   2024-11-08 15:00:00+00:00  0.03  0.03  0.03   0.03   
                   2024-11-08 16:00:00+00:00  0.04  0.04  0.04   0.04   
                   2024-11-08 17:00:00+00:00  0.03  0.03  0.03   0.03   
                   2024-11-11 14:00:00+00:00  0.02  0.02  0.02   0.02   
                   2024-11-11 16:00:00+00:00  0.01  0.01  0.01   0.01   
                   2024-11-11 18:00:00+00:00  0.01  0.02  0.01   0.02   

                                               volume  trade_count   vwap  
symbol             timestamp                                               
SPY241112P00540000 2024-11-07 16:00:00+00:00     50.0          3.0  0.070  
                   2024-11-07 19:00:00+00:00      2.0          2.0  0.060  
                   2024-11-07 20:00:00+00:00      1.0          1.0  0.060  
                   2024-11-08 14:00:00+00:00      1.0          1.0  0.040  
                   2024-11-08 15:00:00+00:00      2.0          2.0  0.030  
                   2024-11-08 16:00:00+00:00      3.0          1.0  0.040  
                   2024-11-08 17:00:00+00:00     10.0          3.0  0.030  
                   2024-11-11 14:00:00+00:00  12501.0          2.0  0.020  
                   2024-11-11 16:00:00+00:00      1.0          1.0  0.010  
                   2024-11-11 18:00:00+00:00      2.0          2.0  0.015

In [22]:
# get options historical trades by symbol
req = OptionTradesRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end=None,                                                     # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_trades(req).df

exchange  price  size  \
symbol             timestamp                                                
SPY241112P00540000 2024-11-07 16:06:49.478416+00:00        B   0.07   6.0   
                   2024-11-07 16:06:49.478416+00:00        B   0.07  24.0   

                                                    conditions  
symbol             timestamp                                    
SPY241112P00540000 2024-11-07 16:06:49.478416+00:00          a  
                   2024-11-07 16:06:49.478416+00:00          a

In [18]:
# get options exchange codes
option_historical_data_client.get_option_exchange_codes()

{'A': 'AMEX - NYSE American',
 'B': 'BOX - Boston Options Exchange',
 'C': 'CBOE - Cboe Options Exchange',
 'D': 'EMERALD - Miami International Stock Exchange Emerald Options',
 'E': 'EDGX - Cboe EDGX Options Exchange',
 'H': 'GEMX - Nasdaq GEMX',
 'I': 'ISE - Nasdaq International Securities Exchange',
 'J': 'MRX - Nasdaq MRX',
 'M': 'MIAX - Miami International Stock Exchange',
 'N': 'NYSE - NYSE Arca',
 'O': 'OPRA - Options Price Reporting Authority',
 'P': 'PEARL - Miami International Stock Exchange Pearl Options',
 'Q': 'NASD - Nasdaq Options',
 'S': 'SPHR - Miami International Stock Exchange Sapphire Options',
 'T': 'BX - Nasdaq BX Options',
 'U': 'MEMX - Members Options Exchange',
 'W': 'C2 - Cboe C2 Options Exchange',
 'X': 'PHLX - Nasdaq PHLX',
 'Z': 'BATS - Cboe BZX Options Exchange'}

In [19]:
# get option latest quote by symbol
req = OptionLatestQuoteRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_quote(req)

{'SPY241112P00540000': {   'ask_exchange': 'N',
     'ask_price': 0.01,
     'ask_size': 1396.0,
     'bid_exchange': '?',
     'bid_price': 0.0,
     'bid_size': 0.0,
     'conditions': ' ',
     'symbol': 'SPY241112P00540000',
     'tape': None,
     'timestamp': datetime.datetime(2024, 11, 11, 21, 13, 50, 518633, tzinfo=TzInfo(UTC))}}

In [23]:
# get option latest trade by symbol
req = OptionLatestTradeRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_trade(req)

{'SPY241112P00540000': {   'conditions': 'I',
     'exchange': 'J',
     'id': None,
     'price': 0.01,
     'size': 44.0,
     'symbol': 'SPY241112P00540000',
     'tape': None,
     'timestamp': datetime.datetime(2024, 11, 11, 19, 42, 6, 809369, tzinfo=TzInfo(UTC))}}

In [24]:
# get option snapshot by symbol
req = OptionSnapshotRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_snapshot(req)

{'SPY241112P00540000': {   'greeks': None,
     'implied_volatility': None,
     'latest_quote': {   'ask_exchange': 'N',
                         'ask_price': 0.01,
                         'ask_size': 1396.0,
                         'bid_exchange': '?',
                         'bid_price': 0.0,
                         'bid_size': 0.0,
                         'conditions': ' ',
                         'symbol': 'SPY241112P00540000',
                         'tape': None,
                         'timestamp': datetime.datetime(2024, 11, 11, 21, 13, 50, 518633, tzinfo=TzInfo(UTC))},
     'latest_trade': {   'conditions': 'I',
                         'exchange': 'J',
                         'id': None,
                         'price': 0.01,
                         'size': 44.0,
                         'symbol': 'SPY241112P00540000',
                         'tape': None,
                         'timestamp': datetime.datetime(2024, 11, 11, 19, 42, 6, 809369, tzinfo=TzInfo(UTC))

In [31]:
# get option chain by underlying_symbol
req = OptionChainRequest(
    underlying_symbol = high_open_interest_contract.underlying_symbol,
)
option_historical_data_client.get_option_chain(req)

KeyboardInterrupt: 

# Market Data (Stream)

In [ ]:
option_data_stream_client = OptionDataStream(api_key, secret_key, url_override = option_stream_data_wss)

async def option_data_stream_handler(data):
    print(data)

symbols = [
    high_open_interest_contract.symbol,
]

option_data_stream_client.subscribe_quotes(option_data_stream_handler, *symbols) 
option_data_stream_client.subscribe_trades(option_data_stream_handler, *symbols)

option_data_stream_client.run()